In [ ]:
# === ETAPA 1: IMPORTAÇÕES E LEITURA DOS DADOS ===
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import json, ast
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

# --- Leitura dos arquivos CSV (sep=";") ---
campaign = pd.read_csv("Campaign_API_ready.csv", sep=";", dtype=str)
cq       = pd.read_csv("CampaignQueue_API_ready.csv", sep=";", dtype=str)
customer = pd.read_csv("Customer_API_ready.csv", sep=";", dtype=str)
order    = pd.read_csv("Order_API_ready.csv", sep=";", dtype=str)

print("Arquivos carregados com sucesso!")
print(f"Campanhas: {campaign.shape}")
print(f"Mensagens: {cq.shape}")
print(f"Clientes: {customer.shape}")
print(f"Pedidos: {order.shape}")


# === ETAPA 2: LIMPEZA E PADRONIZAÇÃO DOS DADOS ===
# 2.1 normaliza nomes de colunas
for df in [campaign, cq, customer, order]:
    df.columns = df.columns.str.strip().str.lower()

# 2.2 conversão robusta de datas (sem format fixo para aceitar ISO com/sem milissegundos)
def parse_datetimes_flex(df):
    for col in df.columns:
        if (("at" in col) or ("date" in col)) and ("status" not in col):
            df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
    return df

campaign = parse_datetimes_flex(campaign)
cq       = parse_datetimes_flex(cq)
customer = parse_datetimes_flex(customer)
order    = parse_datetimes_flex(order)

# 2.3 utilitário seguro para números (evita erro com .str)
def to_num(series):
    """Converte string ou número para float, corrigindo vírgulas decimais."""
    if pd.api.types.is_numeric_dtype(series):
        return series
    return pd.to_numeric(series.astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Ajuste de tipos de telefone
if "customerphone" in customer.columns:
    customer["customerphone"] = customer["customerphone"].astype(str)
if "phonenumber" in cq.columns:
    cq["phonenumber"] = cq["phonenumber"].astype(str)

print(" Dados padronizados e tipos ajustados!")


# === ETAPA 3: DERIVAÇÃO DE DADOS ===
# 3.1 DDD dos clientes (se houver telefone)
if "customerphone" in customer.columns:
    customer["ddd"] = customer["customerphone"].str[2:4]
else:
    customer["ddd"] = np.nan

# 3.2 Indicador de resposta (True/False) em CampaignQueue
cq["response"] = cq.get("response", "").astype(str).str.strip()
cq["tem_resposta"] = cq["response"].apply(lambda x: False if x.lower() in ["nan", "none", "", "null"] else True)

# 3.3 Hora do envio (se existir)
if "sendat" in cq.columns:
    cq["hora_envio"] = cq["sendat"].dt.hour
else:
    cq["hora_envio"] = np.nan

# 3.4 Converte colunas numéricas importantes
if "total.orderamount" in order.columns:
    order["total.orderamount"] = to_num(order["total.orderamount"])
if "preparationtime" in order.columns:
    order["preparationtime"] = to_num(order["preparationtime"])

# 3.5 Cálculo de tempo de preparo (preparationStartDateTime -> deliveryDateTime)
prep_start_col = "preparationstartdatetime" if "preparationstartdatetime" in order.columns else None
delivery_col   = "deliverydatetime" if "deliverydatetime" in order.columns else None

if prep_start_col and delivery_col:
    order["preparationtime_calc"] = (
        order[delivery_col] - order[prep_start_col]
    ).dt.total_seconds() / 60
    order["preparationtime_final"] = order["preparationtime"].fillna(order["preparationtime_calc"])
else:
    order["preparationtime_final"] = order.get("preparationtime")

print("⚙️ Conversão de datas e números finalizada com sucesso!")

# === ETAPA 4: INTEGRAÇÃO DOS DADOS ===
# 4.1 normaliza chaves como string
for col in ["id", "campaignid", "customerid"]:
    if col in cq.columns:       cq[col] = cq[col].astype(str).str.strip()
    if col in campaign.columns: campaign[col] = campaign[col].astype(str).str.strip()
    if col in customer.columns: customer[col] = customer[col].astype(str).str.strip()
    if col in order.columns:    order[col] = order[col].astype(str).str.strip()

# 4.2 merges
dados_integrados = cq.merge(
    campaign, left_on="campaignid", right_on="id", how="left", suffixes=("", "_camp")
)
dados_integrados = dados_integrados.merge(
    customer, left_on="customerid", right_on="id", how="left", suffixes=("", "_cust")
)

# order foi gerado com 'customer.id' como chave do cliente
order_key = "customer.id" if "customer.id" in order.columns else "customerid"
dados_integrados = dados_integrados.merge(
    order, left_on="customerid", right_on=order_key, how="left", suffixes=("", "_order")
)

print("Tabelas integradas com sucesso!")


# === ETAPA 5: FORMATAÇÃO FINAL (dataset analítico compacto) ===
colunas_finais = []
# escolhe colunas se existirem
for c in ["campaignid", "name", "badge", "type", "status",
          "customerid", "customerphone", "ddd", "tem_resposta",
          "hora_envio", "saleschannel", "total.orderamount"]:
    if c in dados_integrados.columns:
        colunas_finais.append(c)

df_final = dados_integrados[colunas_finais].copy() if colunas_finais else dados_integrados.copy()

# renomeia para nomes amigáveis
rename_map = {
    "name": "campanha",
    "badge": "categoria",
    "type": "tipo_campanha",
    "status": "status_campanha",
    "total.orderamount": "valor_pedido"
}
df_final = df_final.rename(columns={k:v for k,v in rename_map.items() if k in df_final.columns})

print("Dados formatados para análise!")


# === ETAPA 6: ANÁLISES ===
# 6.1 Tipos de campanha (mapeia 1/2)
if "type" in campaign.columns:
    camp_copy = campaign.copy()
    camp_copy["type"] = camp_copy["type"].replace({"1":"Promocional","2":"Institucional"})
    tipo_campanha = camp_copy["type"].value_counts(dropna=False).reset_index()
    tipo_campanha.columns = ["Tipo de Campanha", "Quantidade"]
else:
    tipo_campanha = pd.DataFrame({"Tipo de Campanha":[],"Quantidade":[]})

# 6.2 Status das campanhas
status_map = {"1":"Rascunho","2":"Cancelada","3":"Publicada","4":"Concluída"}
campaign["status_legivel"] = campaign.get("status","").map(status_map).fillna("Desconhecido")
status_camp = campaign["status_legivel"].value_counts(dropna=False).reset_index()
status_camp.columns = ["Status","Quantidade"]

# 6.3 Campanhas com maior taxa de resposta
if not df_final.empty and "campanha" in df_final.columns and "tem_resposta" in df_final.columns:
    campanhas_respostas = (
        df_final.dropna(subset=["campanha"])
                .groupby("campanha", dropna=False)["tem_resposta"]
                .mean()
                .reset_index()
    )
    campanhas_respostas["taxa_resposta_%"] = (campanhas_respostas["tem_resposta"]*100).round(2)
    campanhas_respostas = campanhas_respostas.sort_values("taxa_resposta_%", ascending=False)
else:
    campanhas_respostas = pd.DataFrame(columns=["campanha","tem_resposta","taxa_resposta_%"])

print("\n📊 Amostra de campanhas com respostas detectadas:")
print(campanhas_respostas.head(10))


# === ETAPA 7: VISUALIZAÇÕES ===
if not tipo_campanha.empty:
    fig1 = px.pie(
        tipo_campanha, names="Tipo de Campanha", values="Quantidade",
        title="📊 Tipos de Campanha", color_discrete_sequence=px.colors.qualitative.Pastel
    ); fig1.show()

if not status_camp.empty:
    fig2 = px.bar(
        status_camp, x="Status", y="Quantidade",
        title="📌 Status das Campanhas", text="Quantidade",
        color="Status", color_discrete_sequence=px.colors.qualitative.Safe
    ); fig2.update_traces(textposition="outside"); fig2.show()

if not campanhas_respostas.empty:
    top_n = campanhas_respostas.head(10)
    fig3 = px.bar(
        top_n, x="campanha", y="taxa_resposta_%",
        title="💬 Campanhas com Maior Taxa de Resposta (%)",
        text="taxa_resposta_%", color="taxa_resposta_%", color_continuous_scale="Blues"
    ); fig3.update_traces(textposition="outside"); fig3.show()
else:
    print("Nenhuma campanha com respostas válidas para exibir no gráfico.")

# Engajamento por DDD (se DDD existir)
if "ddd" in df_final.columns:
    engajamento_ddd = df_final.groupby("ddd", dropna=False)["tem_resposta"].mean().reset_index()
    engajamento_ddd["taxa_resposta_%"] = (engajamento_ddd["tem_resposta"]*100).round(2)
    fig4 = px.bar(
        engajamento_ddd, x="ddd", y="taxa_resposta_%",
        title="Engajamento por Região (DDD)",
        text="taxa_resposta_%", color="taxa_resposta_%", color_continuous_scale="Teal"
    ); fig4.update_traces(textposition="outside"); fig4.show()


# === ETAPA 8: INDICADORES GERAIS ===
# ticket médio
ticket_medio_geral = np.nan
if "total.orderamount" in order.columns:
    ticket_medio_geral = round(order["total.orderamount"].mean(skipna=True), 2)

# tempo médio bruto (preparationStartDateTime -> deliveryDateTime) OU baseado em preparationTime do CSV
tempo_medio = np.nan
if "preparationtime_final" in order.columns:
    tempo_medio = round(order["preparationtime_final"].mean(skipna=True), 2)

print("\n📈 === INDICADORES GERAIS ===")
print(f"⏱️ Tempo médio de preparo: {tempo_medio} minutos")
print(f"💸 Ticket médio: R$ {ticket_medio_geral}")


# === ETAPA 9: INDICADORES GERAIS ===
print("\n📈 === INDICADORES GERAIS ===")

# Ticket médio e tempo médio de preparo
order["total.orderamount"] = pd.to_numeric(order.get("total.orderamount", 0), errors="coerce")

# Usa a coluna calculada de tempo final
if "preparationtime_final" in order.columns:
    tempo_medio = round(order["preparationtime_final"].mean(skipna=True), 2)
else:
    tempo_medio = round(order.get("preparationtime", pd.Series([np.nan])).mean(skipna=True), 2)

ticket_medio_geral = round(order["total.orderamount"].mean(skipna=True), 2)

print(f"⏱️ Tempo médio de preparo: {tempo_medio if not np.isnan(tempo_medio) else 'N/D'} minutos")
print(f"💸 Ticket médio: R$ {ticket_medio_geral if not np.isnan(ticket_medio_geral) else 'N/D'}")

# === ETAPA 10: INSIGHTS AVANÇADOS ===
print("\n=== ETAPA 10: INSIGHTS AVANÇADOS ===")

def exists(df, cols):
    return all(c in df.columns for c in cols)

# 10.1 KPIs por canal
if exists(order, ["saleschannel", "total.orderamount", "preparationtime_final"]):
    kpis_canal = (order
        .groupby("saleschannel", dropna=False)
        .agg(
            pedidos=("id", "count") if "id" in order.columns else ("total.orderamount","count"),
            receita=("total.orderamount","sum"),
            ticket_medio=("total.orderamount","mean"),
            tempo_medio_preparo=("preparationtime_final","mean")
        )
        .reset_index()
        .sort_values("receita", ascending=False)
    )
    kpis_canal[["ticket_medio","tempo_medio_preparo"]] = kpis_canal[["ticket_medio","tempo_medio_preparo"]].round(2)

    print("\n📊 KPIs por Canal:")
    display(kpis_canal.head(10) if 'display' in globals() else kpis_canal.head(10))

    fig1 = px.bar(kpis_canal, x="saleschannel", y="receita",
                  title="💰 Receita por Canal",
                  text="receita", color="saleschannel")
    fig1.update_traces(texttemplate="R$ %{text:.2f}", textposition="outside")
    fig1.show()

    fig2 = px.bar(kpis_canal, x="saleschannel", y="tempo_medio_preparo",
                  title="⏱️ Tempo Médio de Preparo por Canal (min)",
                  text="tempo_medio_preparo", color="saleschannel")
    fig2.update_traces(texttemplate="%{text:.2f} min", textposition="outside")
    fig2.show()
else:
    print("Não foi possível calcular KPIs por canal.")

# 10.2 KPIs por loja
if "store.name" in order.columns and "total.orderamount" in order.columns:
    kpis_loja = (order
        .groupby("store.name", dropna=False)
        .agg(
            pedidos=("id", "count") if "id" in order.columns else ("total.orderamount","count"),
            receita=("total.orderamount","sum"),
            ticket_medio=("total.orderamount","mean"),
            tempo_medio=("preparationtime_final","mean") if "preparationtime_final" in order.columns else ("total.orderamount","size")
        )
        .reset_index()
        .sort_values("receita", ascending=False)
    )
    kpis_loja[["ticket_medio","tempo_medio"]] = kpis_loja[["ticket_medio","tempo_medio"]].round(2)

    print("\n🏪 KPIs por Loja:")
    display(kpis_loja.head(10) if 'display' in globals() else kpis_loja.head(10))

    fig3 = px.bar(kpis_loja.head(12), x="store.name", y="receita",
                  title="🏆 Top Lojas por Receita",
                  text="receita", color="store.name")
    fig3.update_traces(texttemplate="R$ %{text:.2f}", textposition="outside")
    fig3.update_layout(xaxis_tickangle=-30, showlegend=False)
    fig3.show()
else:
    print("⚠️ Não foi possível calcular KPIs por loja.")

# === 10.3 Heatmap de Engajamento por Hora e Dia da Semana ===
if "sendat" in cq.columns:
    # Extrai hora e dia da semana em português
    cq["hora"] = cq["sendat"].dt.hour
    cq["dia_semana"] = cq["sendat"].dt.day_name()

    # Tradução manual para português (sem depender do locale do sistema)
    dias_pt = {
        "Monday": "Segunda",
        "Tuesday": "Terça",
        "Wednesday": "Quarta",
        "Thursday": "Quinta",
        "Friday": "Sexta",
        "Saturday": "Sábado",
        "Sunday": "Domingo"
    }
    cq["dia_semana"] = cq["dia_semana"].map(dias_pt)

    # Agrupa para calcular média de respostas por hora e dia
    hm = (
        cq.groupby(["dia_semana", "hora"], dropna=False)["tem_resposta"]
        .mean()
        .reset_index()
    )
    hm["taxa_resposta_%"] = (hm["tem_resposta"] * 100).round(1)

    # Ordena os dias de forma cronológica (Seg → Dom)
    ordem_dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]
    hm["dia_semana"] = pd.Categorical(hm["dia_semana"], categories=ordem_dias, ordered=True)
    hm = hm.sort_values(["dia_semana", "hora"])

    # Cria o heatmap
    fig4 = px.density_heatmap(
        hm,
        x="hora",
        y="dia_semana",
        z="taxa_resposta_%",
        title="Engajamento por Hora e Dia da Semana",
        nbinsx=24,
        histfunc="avg",
        color_continuous_scale="Blues",
        labels={"hora": "Hora do Envio", "dia_semana": "Dia da Semana", "taxa_resposta_%": "Taxa de Resposta (%)"}
    )

    fig4.update_layout(
        xaxis_title="Hora do Envio (0–23h)",
        yaxis_title="Dia da Semana",
        coloraxis_colorbar_title="Taxa de Resposta (%)",
        xaxis=dict(dtick=1)
    )
    fig4.show()
else:
    print("Heatmap não gerado: coluna 'sendAt' ausente em CampaignQueue.")


# 10.4 Conversão por campanha
if exists(dados_integrados, ["name","tem_resposta"]) and "total.orderamount" in dados_integrados.columns:
    conv_camp = (dados_integrados
        .groupby("name", dropna=False)
        .agg(
            mensagens=("id","count") if "id" in dados_integrados.columns else ("tem_resposta","count"),
            taxa_resposta=("tem_resposta","mean"),
            ticket_medio=("total.orderamount","mean")
        )
        .reset_index()
    )
    conv_camp["taxa_resposta_%"] = (conv_camp["taxa_resposta"]*100).round(1)
    conv_camp["ticket_medio"] = conv_camp["ticket_medio"].round(2)
    conv_camp = conv_camp.sort_values(["taxa_resposta_%","ticket_medio"], ascending=[False, False])

    print("\n🎯 Conversão por Campanha:")
    display(conv_camp.head(10) if 'display' in globals() else conv_camp.head(10))

    fig5 = px.scatter(conv_camp, x="taxa_resposta_%", y="ticket_medio",
                      size="mensagens", hover_name="name",
                      title="🎯 Campanhas: Taxa de Resposta (%) vs Ticket Médio (R$)")
    fig5.show()
else:
    print("Não foi possível gerar Conversão por Campanha.")

# 10.5 Funil de status de disparos
if "status" in cq.columns:
    status_map = {1:"Scheduled", 2:"Send", 3:"Received", 4:"Read", 5:"Deleted", 6:"Pending"}
    cq["status_legivel"] = cq["status"].map(status_map).fillna("Desconhecido")
    funil = cq["status_legivel"].value_counts(dropna=False).reset_index()
    funil.columns = ["Etapa","Quantidade"]

    print("\n🧭 Funil de Disparos:")
    display(funil if 'display' in globals() else funil)

    fig6 = px.funnel(funil, x="Quantidade", y="Etapa", title="🧭 Funil de Disparos")
    fig6.show()
else:
    print("Funil não gerado (status ausente).")

# 10.6 Impacto de Campanha em Pedidos (janela de 7 dias)
if exists(cq, ["customerid","sendat"]) and exists(order, ["customer.id","createdat","total.orderamount"]):
    base_msgs = cq[["customerid","sendat"]].dropna().rename(columns={"customerid":"cid"})
    base_ped  = order[["customer.id","createdat","total.orderamount"]].dropna().rename(columns={"customer.id":"cid"})
    merged = base_msgs.merge(base_ped, on="cid", how="left")

    within = merged[(merged["createdat"] >= merged["sendat"]) &
                    (merged["createdat"] <= merged["sendat"] + pd.Timedelta(days=7))]

    uplift = within["total.orderamount"].sum()
    pedidos_impacto = within.shape[0]

    print("\n📈 Impacto estimado (janela de 7 dias pós-envio):")
    print(f"- Pedidos atribuídos à campanha: {pedidos_impacto}")
    print(f"- Receita atribuída: R$ {round(uplift,2)}")
else:
    print("Skipping impacto em janela: colunas 'sendAt'/'createdAt' ausentes.")


✅ Arquivos carregados com sucesso!
Campanhas: (30, 17)
Mensagens: (100, 16)
Clientes: (100, 16)
Pedidos: (100, 18)
🧹 Dados padronizados e tipos ajustados!
⚙️ Conversão de datas e números finalizada com sucesso!
🔗 Tabelas integradas com sucesso!
📦 Dados formatados para análise!

📊 Amostra de campanhas com respostas detectadas:
                     campanha  tem_resposta  taxa_resposta_%
14      Semana do Café + Doce      1.000000           100.00
5            Cupom Pós-Evento      1.000000           100.00
8   Experimente Novos Sabores      0.875000            87.50
15         Semana do Pistache      0.809524            80.95
4    Comemore seu Aniversário      0.750000            75.00
2              Cliente do Mês      0.750000            75.00
12        Promoção do Domingo      0.666667            66.67
7   Entrega Grátis via 99Food      0.600000            60.00
6        Cupom de Boas-Vindas      0.583333            58.33
11           Festival do Doce      0.571429            57.14


/tmp/ipython-input-3170155281.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
/tmp/ipython-input-3170155281.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
/tmp/ipython-input-3170155281.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
/tmp/ipython-input-3170155281.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. T


📈 === INDICADORES GERAIS ===
⏱️ Tempo médio de preparo: 64.13 minutos
💸 Ticket médio: R$ 94.27

📈 === INDICADORES GERAIS ===
⏱️ Tempo médio de preparo: 64.13 minutos
💸 Ticket médio: R$ 94.27

=== ETAPA 10: INSIGHTS AVANÇADOS ===

📊 KPIs por Canal:


,saleschannel,pedidos,receita,ticket_medio,tempo_medio_preparo
2,IFOOD,44,4393.18,99.85,67.89
0,99FOOD,22,2352.97,106.95,53.45
4,WHATSAPP,24,1689.01,70.38,66.79
1,DELIVERYVIP,7,701.18,100.17,66.86
3,SITE,3,290.18,96.73,59.67



🏪 KPIs por Loja:


...


🎯 Conversão por Campanha:


,name,mensagens,taxa_resposta,ticket_medio,taxa_resposta_%
14,Semana do Café + Doce,9,1.000000,105.12,100.0
5,Cupom Pós-Evento,8,1.000000,93.38,100.0
8,Experimente Novos Sabores,8,0.875000,115.05,87.5
15,Semana do Pistache,21,0.809524,76.59,81.0
2,Cliente do Mês,4,0.750000,137.46,75.0
4,Comemore seu Aniversário,4,0.750000,72.23,75.0
12,Promoção do Domingo,12,0.666667,94.76,66.7
7,Entrega Grátis via 99Food,5,0.600000,136.59,60.0
6,Cupom de Boas-Vindas,12,0.583333,102.48,58.3
11,Festival do Doce,7,0.571429,115.48,57.1



🧭 Funil de Disparos:


,Etapa,Quantidade
0,Desconhecido,100



📈 Impacto estimado (janela de 7 dias pós-envio):
- Pedidos atribuídos à campanha: 116
- Receita atribuída: R$ 11601.55
